
# 1. Strands를 활용한 Agentic AI 이해 

**참조 사이트** : https://strandsagents.com/

## 1.1 사전 준비사항

- Python 3.8+ 설치
- AWS CLI 설정 및 Bedrock 모델 액세스 권한
- 기본적인 Python 프로그래밍 지식
- AWS 서비스에 대한 기본 이해

## 1.2 AWS Bedrock 설정 확인

1. **AWS 콘솔에서 Bedrock 모델 access 선언**
   - AWS 콘솔(https://console.aws.amazon.com) 접속

2. **AWS 자격 증명 확인**
   - AWS CLI 설치 확인: `aws --version`
   - 자격 증명 설정 확인: `aws configure list`
   - 필요시 자격 증명 설정: `aws configure`

3. **권한 확인**
   - IAM 콘솔에서 사용자/역할에 "bedrock:InvokeModel" 권한이 있는지 확인

## 1.3 환경 설정

먼저 필요한 패키지들을 설치하고 AWS 자격 증명을 설정합니다.

In [ ]:
# 필요한 패키지 설치

pkg_list="strands-agents strands-agents-tools strands-agents-builder boto3 requests"
! pip install --upgrade $pkg_list > /dev/null 2>&1
! pip list | egrep 'strands|boto3|requests' 

## 1.4 프로그램 초기 선언
- 초기 선언및 이 실습에 도움을 제공할 Helper 스크립트 정의

In [ ]:
# 필요한 라이브러리 임포트
import boto3
import json
import time
from datetime import datetime
from typing import Dict, List, Any
from strands import Agent, tool 
from strands.models import BedrockModel

# 프로그램의 시간측정을 위한 Class 선언 
class Timer:
    def __enter__(self):
        self.start = time.time()
        return self

    def __exit__(self, *args):
        print("\n\n------------------------------------------")
        print(f"실행시간: {time.time() - self.start:.2f}초")

# --------------------------------------------------------------------
# 현재 Note BooK에 Role이 선언되어 있는 경우 AWS 자격 증명 확인해야 하는 경우에 선언 
# --------------------------------------------------------------------
try:
    session = boto3.Session()
    credentials = session.get_credentials()
    print("AWS 자격 증명 확인 완료")
    print(f"리전: {session.region_name}")
except Exception as e:
    print(f"AWS 자격 증명 오류: {e}")
    print("AWS CLI를 사용하여 자격 증명을 설정해주세요: aws configure")

# 현재 리전 확인
current_region = boto3.Session().region_name

# --------------------------------------------------------------------
# 직접 AWS 자격 증명 설정해야 하는경우 
# --------------------------------------------------------------------
# aws_access_key_id = ''             # <-- IAM 사용자의 Access key 를 입력한다.
# aws_secret_access_key = ''         # <-- Secret Access key를 입력한다.
# region_name='us-east-1'            # <-- 리전을 선언한다.

# session = boto3.Session(
#     aws_access_key_id=aws_access_key_id,
#     aws_secret_access_key=aws_secret_access_key,
#     region_name=region_name
# )

# --------------------------------------------------------------------
# Bedrock Client 설정
# --------------------------------------------------------------------
bedrock = session.client(service_name='bedrock-runtime')
bedrock2 = session.client('bedrock')

# --------------------------------------------------------------------
# Bedrock Model ID 설정
# --------------------------------------------------------------------

## 직접 모델을 지정할 경우, 리전 정확히 확인함.
# bedrock_model_claude= "us.anthropic.claude-3-7-sonnet-20250219-v1:0"   # claude sonnet 사용할 경우 
# bedrock_model_nova="amazon.nova-lite-v1:0"                             # Nova lite mode 사용할 경우 

# 현재 리전에 맞는 Model ID 자동 검색
models = bedrock2.list_foundation_models()['modelSummaries']

# Titan Embed Text V2
# find_models = next((m for m in models if 'titan-embed-text-v2' in m['modelId'].lower()), None)
# bedrock_model_titan_emb = find_models['modelId']
# print("[Model ID] titan-embed-text-v2: ", bedrock_model_titan_emb)

# Claude 3.5 haiku 의 버지니아 리전에 따른 Mode-ID확인   
if current_region == 'us-east-1':
    bedrock_model_haiku = "us.anthropic.claude-3-5-haiku-20241022-v1:0"
else:
    find_models = next((m for m in models if 'claude-3-5-haiku' in m['modelId'].lower()), None)
    bedrock_model_haiku = find_models['modelId']

# print("[Model ID] claude-3-5-haiku:", bedrock_model_haiku)
# print("[Current Region]:", current_region)

# 이 실습에서 사용하는 기본 모델 선언 
base_model=bedrock_model_haiku


---

## 2. Strand Agent 


### 2.1 Stream & Non-stream 모드 선언 
- Stream은 Agent에 요청하고 응답하는 단계에서 Strem 형식으로 메세지를 확인하는 모드 이며
- Non-stream 모드는 Agent의 응답이 완성이 완료된 메세지만 받음.  

### 2.2 Stream mode Agent 선언및 Agent에 질의 


In [ ]:
# Stream 형식으로 모델 선언 
streaming_bedrock_model = BedrockModel(
    model_id=base_model,
    temperature=0.3,
    boto_session=session,
    streaming=True,  # Stream 형식으로 출력할지 선택 
)

# Agent 생성및 초기화 
stream_agent = Agent(model=streaming_bedrock_model)

# Stream 형식으로 Agent 실행 
with Timer():       # 실행 시간을 출력하기 위해서 사용 
    stream_result=stream_agent("Stream mode로 실행한다는 것이 무엇인가요?")

### 2.2 Non-stream mode Agent 선언및 Agent에 질의 

In [ ]:
# Non-Stream 형식으로 모델 선언 
non_streaming_bedrock_model = BedrockModel(
    model_id=base_model,
    temperature=0.3,
    boto_session=session,
    streaming=False  # Non-Strem 형식으로 출력 
)

# Non-stream Agent 선언  
non_stream_agent = Agent(model=non_streaming_bedrock_model)

# Non-stream 형식으로 Agent 실행 
with Timer(): non_stream_result=non_stream_agent("Stream mode로 반드시 수행해야 하나요?")

### 2.3 callback_handler 를 사용한 메세지 출력 제한
- Agent가 실행중일 때 발생하는 메세지를 화면에 출력하지 않기 위한 방법

In [ ]:
print("[ Agent 실행 ]\n")
non_stream_agent = Agent(model=non_streaming_bedrock_model,callback_handler=None)

with Timer(): non_stream_result=non_stream_agent("Stream mode로 반드시 수행해야 하나요?")

# Agent 실행후에 메세지를 출력함.
print("\n[ Agent 실행 메세지 출력 ]\n")
print(non_stream_result)

### 2.4 System prompt를 함께 사용한 agent 선언및 실행
- 전문가 에이전트를 생성시 이용함

In [ ]:
agent_system = Agent(
    model=streaming_bedrock_model,
    system_prompt="""
    당신은 친근한 AWS 전문가입니다!
    - 한국어로 쉽게 설명해주세요
    - 실용적인 예시를 들어주세요  
    - 이모지를 적절히 사용해주세요
    """
)

with Timer(): agent_system("AWS Lambda에서 VPC내에 RDS와 mysql로 연결하려면 어떻게 해야 할까요?")

### 2.5 채팅 방식으로 카페에서 커피 주문하기
- 입력창이 보이면 요청하는 내용대로 주문을 해보세요. 
- 주문을 완료하면 'quit' 또는 'exit' 를 입려해서 종료하세요!

In [ ]:
system_prompt="""
당신은 커피 주문 접수 전문가입니다. 고객의 주문을 받을 때:

1. 고객의 음료 선호도, 사이즈, 옵션을 정확히 파악하세요
2. 주문 내용에 대한 명확하고 간결한 요약을 제공하세요
3. 각 음료의 특징과 추가 옵션을 상세히 설명하세요
4. 친근하고 전문적인 톤으로 대화하며 체계적으로 정리하세요
5. 최종 주문 확인과 예상 대기시간을 안내하세요
6. 질문은 한가지씩 물어보세요.

서비스 제약사항:
- 실제 결제 정보나 개인정보를 요구하지 마세요
- 알레르기 유발 가능 재료는 반드시 사전 고지하세요
- 항상 고객의 건강상태나 카페인 민감도를 확인하세요
- 미성년자에게는 카페인 함량이 높은 음료 주문 시 주의사항을 안내하세요

주문 처리 절차:
1. 인사 및 메뉴 안내: "안녕하세요! 오늘 어떤 음료를 드시겠어요?"
2. 음료 선택: 기본 음료 종류 확인 (아메리카노, 라떼, 카푸치노 등)
3. 사이즈 확인: Small(S), Medium(M), Large(L) 중 선택
4. 옵션 확인: 샷 추가, 시럽, 우유 종류, 온도 등
5. 알레르기 체크: "혹시 알레르기가 있으신가요?"
6. 주문 요약: "주문하신 내용을 확인해드릴게요..."
7. 결제 및 대기시간 안내: "총 금액은 X원이고, 약 5분 정도 소요됩니다"
"""

agent_system = Agent(
    model=streaming_bedrock_model,
    system_prompt=system_prompt
)

print("무엇을 주문하시겠습니까? \n")
while True:
    user_input = input("고객 (종료시 'quit' 입력)  : ")
    if user_input.lower() in ['quit', 'exit']: break
    response = agent_system(user_input)
    print("\n")
print("감사합니다. 음료가 제조될 때 까지 기다려주세요! \n")

---
## 3. Conversation History(대화기록)

- Agent 는 기본적으로 수명주기 동안 대화내용을 기록하고 있다.이 대화의 기록은 Agent가 수명주기를 다하거나, LLM에 입력되는 토큰수에 제한이 발생하는 이벤트가 발생이되면 오래된 대화내용을 제거하여 대화 내용을 관리하게 된다.
- 이 기본 기능은 대화 기록이 자동으로 유지되어 각 추론 시 모델에 전달되고, 도구 실행 컨텍스트로 사용되며, 컨텍스트 윈
도우 오버플로우를 방지하도록 관리된다.
- 기본 대화 기록 관리 외에도 Conversation Manager를 이용하여 대화내용을 관리할수 도 있다.

### 3.1 모델 초기화

In [ ]:
conversation_bedrock_model = BedrockModel(
    model_id=base_model,
    boto_session=session,
    streaming=True,  # Stream 형식으로 출력할지 선택 
)

### 3.2 Conversation History (대화기록)
- Agent가 기본적으로 대화 내용을 기억하는지 확인 


#### 3.2.1 Agent 생성
- Agent를 생성하면 기본적으로 단기 기억저장소가 생기고 초기화 된다

In [ ]:
agent_memory = Agent(
    model=conversation_bedrock_model,
    callback_handler=None,    
    system_prompt="""
    당신은 라면에 달인 입니다.
    - 한국어로 쉽게 설명해주세요
    - 실용적인 예시를 들어주세요  
    - 이모지를 적절히 사용해주세요
    - 텍스트형식으로 출력해주세요!
    """
)

result=agent_memory("라면에 김치를 넣으면 어떨까요?")
print(f"메세지 내용 : \n{result.message}\n")


- 기억력 테스트를 위해 반복하여 질문해본다.

In [ ]:
result=agent_memory("라면에 파를 넣으면 어떨까요?")
print(f"메세지 내용 : \n{result.message}\n")

#### 3.2.2 Agent에게 기억하고 있는 내용을 물어 본다.

In [ ]:
agent_memory("방금전에 내가 라면에 무엇을 넣어야 하는지 질문한 것들을 알려줘!")

#### 3.2.3 Agent가 기억하고 있는 모든 내용을 출력 한다. 
- 이때 agent.messages 를 사용한다. 이곳에는 모든 사용자와 어시스턴스의 메세지 뿐만 아니라 tool 사용과 결과에 대해서 기록이 이뤄진다.

In [ ]:
# Agent의 모든 기억 내용을 확인 하기 위해 messages 를 확인 한다. 
print(" ********  기억하고 있는 모든 대화 내용을 출력 *************\n")
for item in agent_memory.messages:
    print(item)
print("\n")


#### 3.2.4 Agent가 기억하고 있는 내용중에 질문만 출력해본다.

In [ ]:
for text in agent_memory.messages :
    if text['role'] == 'user' :
        print(text)

### 3.3 Agent 초기화시 강제로 대화기록 삽입
- Agent 초기 설정시 대화기록을 삽입하도록 한다. 
- 이 기능을 이용하면 대화 기록을 장기 저장소에 저장해놓았다가 필요할때 가져와 사용할 수 있도록 할수 있다.

#### 3.3.1 Agent 초기화 하면 기존 대화 기록이 사라지는 것을 확인 

In [ ]:
agent_memory = Agent(
    model=conversation_bedrock_model,
    system_prompt="""
    당신은 라면에 달인 입니다.
    - 한국어로 쉽게 설명해주세요
    - 실용적인 예시를 들어주세요  
    - 이모지를 적절히 사용해주세요
    - 텍스트형식으로 출력해주세요!
    """
)

print(agent_memory.messages)

#### 3.3.2 Agent를 초기화 하면서 대화 기록을 주입 

In [ ]:
# 미리 대화기록을 작성한다.
pre_conv_history=[
    {'role': 'user', 'content': [{'text': '라면에 김치는 넣으면 어떨까요?'}]}, 
    {'role': 'assistant', 'content': [{'text': '# 라면에 김치 넣기 🍜 + 🥬\n\n라면에 김치를 넣는 것은 정말 훌륭한 조합이에요! 김치의 새콤함과 매콤함이 라면의 깊은 맛을 한층 더 끌어올려줍니다. 😋\n\n## 김치 라면의 장점 ✨\n- 김치의 발효된 맛이 라면 국물에 깊이를 더해줍니다\n- 아삭한 식감이 라면의 부드러운 면과 대비되어 식감이 풍부해져요\n- 김치에 있는 유산균이 라면의 건강 밸런스를 약간 개선해줍니다\n\n## 실용적인 팁 💡\n\n1. **신김치 vs 묵은지**\n   - 신김치: 아삭한 식감과 상큼한 맛을 원할 때 (국물 라면에 좋아요)\n   - 묵은지: 깊은 감칠맛을 원할 때 (볶음라면에 특히 좋습니다)\n\n2. **넣는 타이밍**\n   - 끓는 물에 면을 넣기 직전에 김치를 함께 넣으면 김치 맛이 국물에 잘 우러나요\n   - 볶음라면은 면을 거의 다 볶은 후 마지막에 김치를 넣어 살짝만 볶아주세요\n\n3. **추천 조합 예시** 🌟\n   - 신라면 + 잘 익은 배추김치 + 파 + 계란\n   - 진라면 + 묵은지 + 베이컨 조각 + 치즈 한 장\n   - 짜파게티 + 볶은 김치 (김치 볶음면의 정석!)\n\n김치 라면은 한국인의 소울푸드 조합이라고 할 수 있어요. 특히 해장이 필요할 때 더욱 빛을 발하는 조합입니다! 🔥'}]}   
]

# agent 생성시 대화 기록을 전달한다. 
agent_memory = Agent(
    model=conversation_bedrock_model,
    messages=pre_conv_history,
    system_prompt="""
    당신은 라면에 달인 입니다.
    - 한국어로 쉽게 설명해주세요
    - 실용적인 예시를 들어주세요  
    - 이모지를 적절히 사용해주세요
    - 텍스트형식으로 출력해주세요!
    """
)

# 대화 내용을 기억하고 있는지 확인 
agent_memory("방금전에 내가 라면에 무엇을 넣어야 하는지 질문한 것들을 알려줘!")

---
## 4. Tool(도구) 사용 
도구는 에이전트 기능을 확장하는 주요 메커니즘으로, 단순한 텍스트 생성을 넘어서 작업을 수행할 수 있게 해줍니다. 도구를 통해 에이전트는 외부 시스템과 상호작용하고, 데이터에 액세스하며, 환경을 조작할 수 있습니다.

- 참조: https://strandsagents.com/latest/documentation/docs/user-guide/concepts/tools/example-tools-package/



### 4.1 모델 초기화

In [ ]:

tooluse_bedrock_model = BedrockModel(
    model_id=base_model,
    boto_session=session,
    streaming=True,  # Stream 형식으로 출력할지 선택 
)

### 4.2 내장 Tool 구성 및 활용

#### 4.2.1 Tool 사용 승인 선언

일부 Tool 실행시 사용자 승인에 관련된 요청, 예를 들어 파일 저장관련 요청을 할경우 사용자의 입력 요청 ( Do you want to proceed with the file write? [y/*] ) 과 같은 처리시 사용자 요청을 자동처리 하기 위한 방법으로 운영체제 환경에 'BYPASS_TOOL_CONSENT' 값을 true로 선언한다.

In [ ]:
import os

os.environ["BYPASS_TOOL_CONSENT"] = "false"

#### 4.2.2 계산기 도구 사용

In [ ]:
from strands_tools import calculator

# Agent tool 선언 
tooluse_agent = Agent(
    model=tooluse_bedrock_model,
    tools=[calculator]
)

# Agent실행 
tooluse_agent("42 ^ 9 값을 무엇이야? 한국어로 대답해줘!")

print(f"\n실행한 도구 이름: {tooluse_agent.tool_names}")



#### 4.2.3 WEB URL 검색 도구 사용

In [ ]:
from strands_tools import http_request

# Agent tool 선언 
tooluse_agent = Agent(
    model=tooluse_bedrock_model,
    tools=[http_request]
)

# Agent실행 
tooluse_agent("다음 https://docs.aws.amazon.com/ko_kr/bedrock/latest/userguide/model-access.html 사이트에 접속해서 정리 해줘!")



#### 4.2.3 여러 도구들 사용 - 파일 관리

In [ ]:
from strands_tools import file_read, file_write,current_time
import os

os.environ["BYPASS_TOOL_CONSENT"] = "true"

# Agent tool 선언 
tooluse_agent = Agent(
    model=tooluse_bedrock_model,
    tools=[file_read, file_write,current_time]
)

# tool사용을 요청하는 질의 
tooluse_agent("현재 디렉토리 아래에 tooluse_agent.dat 파일이 있는지 확인해주고, 파일이 없으면 파일을 생성하고, 파일에 현재 서울 시간을 기록해줘!")

print(f"\n실행한 도구 이름: {tooluse_agent.tool_names}")
os.environ["BYPASS_TOOL_CONSENT"] = "false"

### 4.3 python function 방식의 tool 생성
- tool 을 function으로 설정함.

#### 4.3.1 가상 날씨 검색 function 생성

In [ ]:
from strands import tool

@tool
def weather_forecast(city: str, days: int = 3) -> str:
    """도시의 날씨 예보 정보 가져옴.
   
    Args:
        city: 도시의 이름
        days: 날씨 예보 기간
    """
    return f"{city} 의  {days} 일간에 날씨는 매우 맑습니다."



#### 4.3.2 생성된 tool 사용


In [ ]:
# Agent tool 선언 
tooluse_agent = Agent(
    model=tooluse_bedrock_model,
    tools=[weather_forecast]
)

# Agent실행 
tooluse_agent("서울의 7일간에 날씨는?")

print(f"\n실행한 도구 이름: {tooluse_agent.tool_names}")


#### 4.3.3 난수발생 function들 생성

In [ ]:
# 소숫점 발생 난수 발생 tool
@tool
def random_float(min_value: float = 0.0, max_value: float = 1.0, precision: int = 2) -> str:
    """지정된 범위에서 실수 난수를 생성합니다.

    Args:
        min_value: 최솟값
        max_value: 최댓값
        precision: 소수점 자릿수
    """
    import random

    result = round(random.uniform(min_value, max_value), precision)
    return f"{min_value}~{max_value} 범위에서 생성된 실수 난수: {result}"

# 문자열 난수 발생 tool
@tool
def random_string(length: int = 8, include_numbers: bool = True, include_symbols: bool = False) -> str:
    """랜덤 문자열을 생성합니다.

    Args:
        length: 문자열 길이
        include_numbers: 숫자 포함 여부
        include_symbols: 특수문자 포함 여부
    """
    import random
    import string

    chars = string.ascii_letters
    if include_numbers:
        chars += string.digits
    if include_symbols:
        chars += "!@#$%^&*"

    result = ''.join(random.choice(chars) for _ in range(length))
    return f"길이 {length}의 랜덤 문자열: {result}"


#### 4.3.4 생성된 tool들 사용


In [ ]:
# Agent tool 선언 
tooluse_agent = Agent(
    model=tooluse_bedrock_model,
    tools=[weather_forecast,random_float,random_string]
)

# Agent실행 
tooluse_agent("1에서 10까지 난수를 만들어주고, 8문자의 숫자와 특수문자가 포함된 문자를 생성해줘!")
print(f"\n실행한 도구 이름: {tooluse_agent.tool_names}")


### 4.4 Direct tool 호출
- Agent에게 어떤 tool을 사용할지 직접 제안하는 방식으로 대화기록에 포함하는 방식과 그렇지 않는 방식으로 호출이 가능함.

#### 4.4.1 대화 기록에 포함하면서 직접 도구 호출
- 도구 실행의 결과를 대화기록에 포함하도록 실행함

In [ ]:
from strands_tools import calculator

tooluse_agent = Agent(
    model=tooluse_bedrock_model,
    tools=[calculator]
)

tooluse_agent.tool.calculator(expression="42 ^ 9")

print(f"\n실행한 도구 이름: {tooluse_agent.tool_names}")
print(f"\n대화기록: {tooluse_agent.messages}")
tooluse_agent("\n계산한 결과값이 뭐지?")


#### 4.5.2 대화 기록에 포함 하지 않으면서 직접 도구 호출
- 도구 실행의 결과를 대화기록에 포함하지 않도록 실행함

In [ ]:
from strands_tools import calculator

tooluse_agent = Agent(
    model=tooluse_bedrock_model,
    tools=[calculator]
)

tooluse_agent.tool.calculator(expression="42 ^ 9" , record_direct_tool_call=False)

print(f"\n실행한 도구 이름: {tooluse_agent.tool_names}")
print(f"\n대화기록: {tooluse_agent.messages}")
tooluse_agent("\n계산한 결과값이 뭐지?")


---
## 5. MCP tool 
- https://awslabs.github.io/mcp/servers/aws-documentation-mcp-server 참조

### 5.1 MCP용 모델 선언

In [ ]:

mcp_bedrock_model = BedrockModel(
    model_id=base_model,
    boto_session=session
)

### 5.2 Streamable HTTP 방식의 MCP서버

#### 5.2.1 AWS 공식 문서 MCP 서버 연결  

In [ ]:
from mcp.client.streamable_http import streamablehttp_client
from strands.tools.mcp.mcp_client import MCPClient 

streamable_http_mcp_client = MCPClient(lambda: streamablehttp_client("https://knowledge-mcp.global.api.aws"))


#### 5.2.2 Agent 선언및 질의

In [ ]:

with streamable_http_mcp_client:
    # Get the tools from the MCP server
    tools = streamable_http_mcp_client.list_tools_sync()
    
    mcp_agent = Agent(model=mcp_bedrock_model,tools=tools)
    mcp_agent("Amazon cognito가 무엇이야? 그리고 문서 URL위치도 알려줘!")
    

## 6. 멀티 에이전트 - WorkFlow

---
### 6.1 WorkFlow

#### 6.1.1 순서적으로 여러 에이전트 실행 
```mermaid
graph LR
    A[조사 에이전트] --> B[분석 에이전트]
    B --> C[보고서 에이전트]
    
    style A fill:#1a1a1a,stroke:#00ff00,stroke-width:2px,color:#ffffff
    style B fill:#1a1a1a,stroke:#00ff00,stroke-width:2px,color:#ffffff
    style C fill:#1a1a1a,stroke:#00ff00,stroke-width:2px,color:#ffffff
```

In [ ]:

# 전문가 에이전트들 생성 
researcher_agent = Agent(system_prompt="당신은 연구 전문가입니다. 주요 정보를 찾아보세요.", model=base_model, callback_handler=None)
analyst_agent = Agent(system_prompt="연구 데이터를 분석하고 인사이트를 추출합니다.",model=base_model, callback_handler=None)
writer_agent = Agent(system_prompt="분석을 바탕으로 세련된 보고서를 작성합니다.",model=base_model,callback_handler=None)

topic = "짜장면"
# Step 1: Research
research_prompt=f"{topic} 에 대한 최신 개발 사항 연구"
print("\n [ 1. researcher agent 실행 중 ... ]\n ")
research_results = researcher_agent(research_prompt)

# Step 2: Analysis
analysis_prompt=f"이 연구 결과를 분석하십시오: {research_results}"
print("\n [ 2. analyst agent 실행 중 ... ]\n ")
analysis = analyst_agent(analysis_prompt)

# Step 3: Report writing
report_prompt=f"이 분석을 바탕으로 보고서를 작성합니다: {analysis}"
print("\n [ 3. writer agent 실행 중... ]\n")
final_report = writer_agent(report_prompt)

print("\n -- 최종 보고서 --\n")
print(final_report)

---
## 7. 멀티 에이전트 - Graph 

Graph 패턴의 동작 원리

Graph 패턴은 체계적이고 예측 가능한 작업 흐름을 기반으로 합니다:

🔹 Node: 각각의 작업자(에이전트)나 처리 단위를 의미합니다  
🔹 Edge: 작업자들 사이의 순서와 정보 전달 경로를 나타냅니다   
🔹 실행 순서: 의존 관계에 따라 자동으로 올바른 순서가 결정됩니다   
🔹 정보 흐름: 앞 단계의 결과가 다음 단계의 재료가 됩니다   
🔹 시작점(EntryPoint): 최초 요청을 받아서 전체 과정을 시작합니다

### 7.1 짜장면 주문 예제로 구성

```mermaid
 graph TD
    Start["🍜 고객 주문 접수"] --> MenuResearch["📋 메뉴 조사 에이전트<br/>menu_researcher<br/>고객 취향 분석 및<br/>최적 짜장면 추천"]
    
    MenuResearch --> TasteAnalysis["👅 맛 분석 에이전트<br/>taste_analyst<br/>면발, 춘장, 토핑<br/>조화 분석"]
    
    MenuResearch --> QualityCheck["✅ 품질 검증 에이전트<br/>quality_checker<br/>신선도 및 조리상태<br/>품질 확인"]
    
    TasteAnalysis --> OrderWriter["📝 주문서 작성 에이전트<br/>order_writer<br/>완벽한 주문서<br/>최종 작성"]
    
    QualityCheck --> OrderWriter
    
    OrderWriter --> Complete["🎉 짜장면 주문 완료!"]
    
    %% 스타일링
    classDef startEnd fill:#ff6b6b,stroke:#ffffff,stroke-width:2px,color:#ffffff
    classDef agent fill:#1a1a1a,stroke:#00ff00,stroke-width:2px,color:#ffffff
    classDef process fill:#4ecdc4,stroke:#ffffff,stroke-width:2px,color:#000000
    
    class Start,Complete startEnd
    class MenuResearch,TasteAnalysis,QualityCheck,OrderWriter agent


```

#### 7.1.1 전문 에이전트 생성

In [ ]:
import logging
from strands.multiagent import GraphBuilder

# 짜장면 주문 시스템 디버그 로그 활성화 🍜
logging.getLogger("strands.multiagent").setLevel(logging.DEBUG)
logging.basicConfig(format="🍜 %(levelname)s | %(name)s | %(message)s",
                   handlers=[logging.StreamHandler()])

# 짜장면 전문 에이전트들 생성 👨‍🍳
menu_researcher = Agent(
    name="menu_researcher", 
    model=base_model,
    system_prompt="당신은 짜장면 메뉴 전문가입니다. 고객 취향에 맞는 최적의 짜장면을 추천합니다..."
)

taste_analyst = Agent(
    name="taste_analyst", 
    model=base_model,
    system_prompt="당신은 짜장면 맛 분석 전문가입니다. 면발, 춘장, 토핑의 조화를 분석합니다..."
)

quality_checker = Agent(
    name="quality_checker", 
    model=base_model,
    system_prompt="당신은 짜장면 품질 검증 전문가입니다. 신선도와 조리 상태를 확인합니다..."
)

order_writer = Agent(
    name="order_writer", 
    model=base_model,
    system_prompt="당신은 짜장면 주문서 작성 전문가입니다. 완벽한 주문서를 작성합니다..."
)


#### 7.1.2 Graph Node와 Edge 구성

In [ ]:

# 짜장면 주문 워크플로우 구성 🏗️
builder = GraphBuilder()

# 각 단계별 노드 추가
builder.add_node(menu_researcher, "menu_research")    # 메뉴 조사
builder.add_node(taste_analyst, "taste_analysis")     # 맛 분석
builder.add_node(quality_checker, "quality_check")    # 품질 검증
builder.add_node(order_writer, "order_writing")       # 주문서 작성

# 짜장면 주문 프로세스 흐름 설정 🔄
builder.add_edge("menu_research", "taste_analysis")   # 메뉴 조사 → 맛 분석
builder.add_edge("menu_research", "quality_check")    # 메뉴 조사 → 품질 검증
builder.add_edge("taste_analysis", "order_writing")   # 맛 분석 → 주문서 작성
builder.add_edge("quality_check", "order_writing")    # 품질 검증 → 주문서 작성


#### 7.1.3 메뉴조사 시작

In [ ]:

# 메뉴 조사부터 시작! 🚀
builder.set_entry_point("menu_research")

# 짜장면 주문 시스템 완성! 🎯
jjajangmyeon_system = builder.build()

# 고객 주문 처리 실행 🍽️
result = jjajangmyeon_system(
    "매운맛을 좋아하고 면발은 쫄깃한 걸 선호하는 고객을 위한 "
    "최고의 짜장면을 추천하고 완벽한 주문서를 작성해주세요"
)

# 주문 결과 확인 📋
print(f"\n🍜 주문 상태: {result.status}")
print(f"🔄 처리 순서: {[node.node_id for node in result.execution_order]}")
print("🎉 맛있는 짜장면 주문이 완료되었습니다!")

# 기존 Logging 레벨을 높임.
logging.getLogger("strands.multiagent").setLevel(logging.CRITICAL)


#### 7.1.4 실행 결과 

In [ ]:

# Check execution status
print(f"Status: {result.status}")  # COMPLETED, FAILED, etc.

# See which nodes were executed and in what order
for node in result.execution_order:
    print(f"Executed: {node.node_id}")

# Get results from specific nodes
taste_analysis_result = result.results["taste_analysis"].result
print(f"taste_analysis: {taste_analysis_result}")

# Get performance metrics
print(f"Total nodes: {result.total_nodes}")
print(f"Completed nodes: {result.completed_nodes}")
print(f"Failed nodes: {result.failed_nodes}")
print(f"Execution time: {result.execution_time}ms")
print(f"Token usage: {result.accumulated_usage}")

### 7.2 조건부 Edge 추가 
- 조건을 추가해서 분기하도록 선언

```mermaid
graph TD
    Start["🍜 고객 주문 접수"] --> MenuResearch["📋 메뉴 조사 에이전트<br/>menu_researcher<br/>고객 취향 분석 및<br/>최적 짜장면 추천"]
    
    MenuResearch -->|"재고 확인"| StockCheck{{"📦 재고 확인<br/>only_if_menu_available"}}
    
    StockCheck -->|"재고 있음"| TasteAnalysis["👅 맛 분석 에이전트<br/>taste_analyst<br/>면발, 춘장, 토핑<br/>조화 분석"]
    
    StockCheck -->|"재고 없음"| OutOfStock["❌ 재고 부족<br/>다른 메뉴 추천"]
    
    MenuResearch --> QualityCheck["✅ 품질 검증 에이전트<br/>quality_checker<br/>신선도 및 조리상태<br/>품질 확인"]
    
    TasteAnalysis --> OrderWriter["📝 주문서 작성 에이전트<br/>order_writer<br/>완벽한 주문서<br/>최종 작성"]
    
    QualityCheck --> OrderWriter
    
    OrderWriter --> Complete["🎉 짜장면 주문 완료!"]
    
    %% 스타일링
    classDef startEnd fill:#ff6b6b,stroke:#ffffff,stroke-width:2px,color:#ffffff
    classDef agent fill:#1a1a1a,stroke:#00ff00,stroke-width:2px,color:#ffffff
    classDef condition fill:#ffd93d,stroke:#ffffff,stroke-width:2px,color:#000000
    classDef failure fill:#ff6b6b,stroke:#ffffff,stroke-width:2px,color:#ffffff
    
    class Start,Complete startEnd
    class MenuResearch,TasteAnalysis,QualityCheck,OrderWriter agent
    class StockCheck condition
    class OutOfStock failure

```

In [ ]:
def only_if_menu_available(state):
    """메뉴가 재고 있을 때만 다음 단계로 진행합니다."""
    menu_node = state.results.get("menu_research")
    if not menu_node:
        return False
    
    # 메뉴 조사 결과에서 재고 확인 지표를 체크
    result_text = str(menu_node.result)
    return "재고있음" in result_text or "available" in result_text.lower()

# 조건부 엣지 추가 - 메뉴 재고가 있을 때만 맛 분석 진행
builder.add_edge("menu_research", "taste_analysis", condition=only_if_menu_available)


### 7.3 Graphs as a Tool 을 이용한 Dynamic 방식의 Agent들 실행

In [ ]:
from strands_tools import graph

# 짜장면 R&D 시스템
jjajangmyeon_rnd = Agent(
    tools=[graph],
    model=base_model,
    system_prompt="짜장면 연구개발을 위한 체계적인 에이전트 워크플로우를 구축합니다."
)

jjajangmyeon_rnd(
    "고객 만족도 95% 이상의 프리미엄 짜장면을 개발하고, "
    "재료 조달부터 최종 서빙까지의 완전한 프로세스를 설계해주세요"
)